In [ ]:
#pip install sklearn

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import pandas as pd
import numpy as np
import panel as pn

from sklearn import datasets, linear_model
from sklearn.metrics import r2_score

In [ ]:
pn.extension('vega')

In [ ]:
data = datasets.load_diabetes()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['Target'] = data.target
df.head()

In [ ]:
X = df.drop('Target', axis=1)
y = df.Target

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(X, y)

y_pred = lr.predict(X)
r2 = r2_score(y, y_pred)
r2

In [ ]:
def get_regression_line(df):
    lr = linear_model.LinearRegression()
    lr.fit(X, y)
    coef = lr.coef_
    intercept = lr.intercept_
    
    scatter_plot = alt.Chart(df).mark_circle().encode(
        x='bmi',
        y='Target',
        tooltip=['bmi', 'Target'])
    
    line_plot = alt.Chart(pd.DataFrame({'bmi': [df['bmi'].min(), 
                                    df['bmi'].max()], 'Target': [df['bmi'].min() * coef[2] + intercept, 
                                    df['bmi'].max() * coef[2] + intercept]})).mark_line(color='red').encode(
        x='bmi', y='Target')
    return scatter_plot + line_plot

In [ ]:
get_regression_line(df)

In [ ]:
def create_box_plot(df):
    return alt.Chart(df).mark_boxplot().encode(
    x=alt.X('key:N', title='Feature'),
    y=alt.Y('value:Q', title='Value'),).transform_fold(list(df.columns[:-1]),
                                                    as_=['key','value']).interactive()

In [ ]:
create_box_plot(df)

In [ ]:
def create_scatter_plot(df):
    return alt.Chart(df).mark_circle().encode(x='bmi', y='Target', tooltip=['bmi','Target'])

def scatter_plot(x_feature, y_feature):
    chart = alt.Chart(df).mark_circle(size=60).encode(
    x=x_feature, y=y_feature, color='Target',
    tooltip=[x_feature, y_feature, 'Target']).properties(width=600, height=400)
    return chart

In [ ]:
x_select = pn.widgets.Select(options=list(df.columns), name='X-axis')
y_select = pn.widgets.Select(options=list(df.columns), name='Y-axis')

@pn.depends(x_select.param.value, y_select.param.value)
def update_plot(x_feature, y_feature):
    return scatter_plot(x_feature, y_feature)

In [ ]:
dashboard = pn.Column(
    pn.pane.Markdown('# Diabetes Regression Analysis Dashboard'),
    pn.Row(get_regression_line(df), create_box_plot(df)),
    pn.Column(x_select, y_select), update_plot)

In [ ]:
dashboard

In [ ]:
# Create the Altair charts
boxplot = alt.Chart(df).mark_boxplot().encode(
    x=alt.X('key:N', title='Feature'),
    y=alt.Y('value:Q', title='Value'),
).transform_fold(
    list(df.columns[:-1]),
    as_=['key', 'value']
).interactive()

scatterplot = alt.Chart(df).mark_circle().encode(
    x=alt.X(alt.repeat("column"), type='quantitative'),
    y=alt.Y('target:Q'),
    tooltip=list(df.columns)
).properties(width=150, height=150).repeat(column=list(df.columns[:-1])).interactive()

linear_regression = alt.Chart(pd.DataFrame({'y': y, 'y_pred': y_pred})).mark_point().encode(
    x=alt.X('y:Q', title='True Target Values'),
    y=alt.Y('y_pred:Q', title='Predicted Target Values'),
    tooltip=['y', 'y_pred']
).interactive()



In [ ]:
dashboard2 = pn.Column(
    pn.pane.Markdown('# Diabetes Regression Dashboard'),
    pn.Row(boxplot, scatterplot),
    pn.Row(linear_regression),
    pn.Row(r2),)

In [ ]:
dashboard2.servable()